ANALITICO DE CUSTO - PROJETO BIO-STG

In [22]:
import sys
import os
import pandas_gbq as pgbq
import pandas as pd
#from google.oauth2 import service_account
import google.auth


In [23]:
def get_file_local(sql):

    # substituido ler aruqivo local, nao gera o arquivo, ler no storage
    path_airflow = os.getcwd()
    scrp_path = f'{path_airflow}/sql/{sql}'

    ## arquivo sql dentro da esrutura airflow: 
    f = open(scrp_path, 'r')

    conteudo = f.readlines()
    # initialize an empty string
    strsql = ""

    # converte arquivo em string
    for ln in conteudo:
        strsql += ln   

    return strsql

In [24]:
user = os.getlogin()
key = f'/home/{user}/.ssh/airflow-bio-sheduler-jobs.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key
#credencial = service_account.Credentials.from_service_account_file(key)
credencial, pid = google.auth.default(     
    scopes=[
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/bigquery",
    "https://www.googleapis.com/auth/cloud-platform"
])

EXTRAIR DADOS DA TABELA DE MONITORIA


In [25]:
#substituir metodo busca sql local: '/home/airflow/dags/transporte/scripts'
sql = 'estudo_analitico_do_custo.sql'
strsql = get_file_local(sql)  
# strsql = geraScriptSql(f'{path}/query_cust/QUERY_ANALISE_CUSTO_TABELAS.sql',None,None) #,value[1])
#GERAR DATAFRAME
df = pgbq.read_gbq(strsql, project_id='composed-night-232419' , credentials=credencial, progress_bar_type=None)

In [26]:
import datetime
current_dt = datetime.date.today() #datetime.date(2023,5,16)

Inicio da analise - consulta de informações 7dias

In [27]:
# extrair dados 7dias
current_dt_d7 = current_dt - datetime.timedelta(7)
# df_all['total_mes'] = df_all.groupby(['data_carga'])['custo_dia_mes'].transform('sum')
# df_all = df_all[['data_carga','total_dia','total_mes']].drop_duplicates()
#df_d0['table_id'].count()
#df_d0['table_id'].sum()
df_project = df.loc[(df['data_carga']>=current_dt_d7)]
df_project = df_project[['project_id','data_carga','qtde_execucao_dia','table_id','custo_dia','custo_dia_mes']]
df_project.groupby(['project_id','data_carga']).agg({'custo_dia':'sum','custo_dia_mes':'sum','table_id':'count', 'qtde_execucao_dia':'sum'})


custo_dia  custo_dia_mes  table_id  \
project_id            data_carga                                       
composed-night-232419 2023-07-17      48.93         1467.9        53   
                      2023-07-18      49.74         1492.2        53   
                      2023-07-19      43.93         1317.9        53   
                      2023-07-20      45.67         1370.1        53   
                      2023-07-21      43.64         1309.2        53   
                      2023-07-22      39.54         1186.2        53   
                      2023-07-23       8.37          251.1        53   
                      2023-07-24       9.92          297.6        53   
                      2023-07-25       0.00            0.0        53   

                                  qtde_execucao_dia  
project_id            data_carga                     
composed-night-232419 2023-07-17               1450  
                      2023-07-18               1414  
                      2023-07-19               1234  
                      2023-07-20               1234  
                      2023-07-21               1234  
                      2023-07-22               1234  
                      2023-07-23                278  
                      2023-07-24                239  
                      2023-07-25                  0

In [28]:
# extrair dados do dia corrente, caso retorne 0 não ha dados, então teve algum problema
current_dt_d0 = current_dt - datetime.timedelta(0)

df_d0=df.loc[(df['data_carga']==current_dt_d0) & (df['qtde_execucao_dia'].notnull())]

df_d0=df_d0[['dataset_id','table_id','prefix_projeto','executor_job_dia','qtde_execucao_dia', 'custo_ano_bee','custo_dia', 'custo_dia_mes','ulti_data_hora_execucao_job_dia']].sort_values('custo_dia_mes', ascending=False)
df_d0.reset_index(inplace=True , drop=True)#
df_d0

,dataset_id,table_id,prefix_projeto,executor_job_dia,qtde_execucao_dia,custo_ano_bee,custo_dia,custo_dia_mes,ulti_data_hora_execucao_job_dia
0,bio_reports,report_bio_o2o_pedidos_abertos_d0,o2o,airflow,9,0.09,2.34,70.2,2023-07-24 11:02:23.764
1,bio_op_instancia_forcada,instancia_forcada_delivery,instancia_forçada,airflow,1,0.61,1.54,46.2,2023-07-24 06:00:07.327
2,bio_op_bob2,bob2_pedido_de_venda_detalhes,o2o,airflow,9,321.90,1.21,36.3,2023-07-24 11:00:34.528
3,bee_analytics_transporte,information_executor_jobs,monitoria,manual,6,0.40,1.13,33.9,2023-07-24 11:10:34.985
4,bio_reports,report_o2o_performance_entrega,o2o,airflow,9,5.10,0.63,18.9,2023-07-24 11:01:53.608
5,bio_o2o_data,report_bio_o2o_funil_cancelamento,o2o,airflow,3,11.36,0.56,16.8,2023-07-24 05:18:55.964
6,bee_analytics_transporte_pipeline_o2o,bob_rastreamento,o2o,airflow,9,5.72,0.45,13.5,2023-07-24 11:00:43.570
7,bee_analytics_transporte_pipeline_o2o,utm_rastreamento,o2o,airflow,9,6.40,0.36,10.8,2023-07-24 11:00:32.247
8,bio_op_bob2,bob2_pedido_de_venda_cabecalho,o2o,airflow,9,299.92,0.36,10.8,2023-07-24 11:00:34.556
9,bee_analytics_transporte_monitoria,historico_info_tabelas,monitoria,manual,30,0.01,0.30,9.0,2023-07-24 11:12:27.517


In [29]:

if len(df_d0)>0:
    geral = df_d0['custo_dia'].sum()
    execucao = df_d0['qtde_execucao_dia'].sum()
    print('Monitoria rodou hoje com sucesso!!!')
    print(f'Valor total dos jobs ao dia ate momento: R$ {int(geral)*5} / total de execuções:{execucao}')

elif len(df_d0)==0:
    print('Monitoria não rodou hoje verificar DAG: !!!')


Monitoria rodou hoje com sucesso!!!
Valor total dos jobs ao dia ate momento: R$ 45 / total de execuções:239


In [30]:
#REPORTS_O2O
bio_reports = ['bio_reports','bio_o2o_data']
tabela_reports_d0 = ['bio_pedido_o2o_eventos_ameflash_sfs','report_bio_o2o_funil_cancelamento','report_bio_o2o_perf_item','report_o2o_performance_entrega','report_bio_o2o_torre_controle','report_bio_o2o_pedidos_aging','report_bio_o2o_pedidos_abertos_d0','report_bio_o2o_lojas_perf_aberto','report_bio_o2o_lojas_perf_aberto_vTorreControle','report_bio_o2o_performance','report_bio_o2o_lojas_perf_aberto_plj']

df_day_reports = df_d0.loc[(df_d0['dataset_id'].isin(bio_reports)) &  (df_d0['table_id'].isin(tabela_reports_d0))]
df_day_reports[['dataset_id','table_id','prefix_projeto','qtde_execucao_dia', 'custo_ano_bee','custo_dia', 'custo_dia_mes','ulti_data_hora_execucao_job_dia']].sort_values('custo_dia_mes', ascending=False)
df_day_reports.reset_index(inplace=True , drop=True)#


In [31]:
#tabela com maior custo mes do grupo reports o2o
df_day_reports[df_day_reports['custo_dia_mes']==df_day_reports['custo_dia_mes'].max()]

,dataset_id,table_id,prefix_projeto,executor_job_dia,qtde_execucao_dia,custo_ano_bee,custo_dia,custo_dia_mes,ulti_data_hora_execucao_job_dia
0,bio_reports,report_bio_o2o_pedidos_abertos_d0,o2o,airflow,9,0.09,2.34,70.2,2023-07-24 11:02:23.764


In [32]:
#CUSTO DETALHE DO GRUPO REPORTS O2O
total_day = df_day_reports[['custo_dia','custo_dia_mes']].sum()

print(f'total em $:\n{total_day}\n')
print(f'total em R$:\n{total_day*5}\n')
print(f'total em R$ no ano:\n{(total_day*5)*12}\n')

#ANALISE EM RELAÇÃO AO PERCENTUAL DE FATIA DO CUSTO DO PROJETO EM COMPARAÇÃO COM GERAL
reports = df_day_reports['custo_dia'].sum()
porc = (reports*100)/df_d0['custo_dia'].sum()
print(f'custo do REPORTS O2O representa {int(porc)}% do custo geral dos jobs executados no dia')

total em $:
custo_dia          3.74
custo_dia_mes    112.20
dtype: float64

total em R$:
custo_dia         18.7
custo_dia_mes    561.0
dtype: float64

total em R$ no ano:
custo_dia         224.4
custo_dia_mes    6732.0
dtype: float64

custo do REPORTS O2O representa 37% do custo geral dos jobs executados no dia


In [33]:
#PIPELINE O2O

o2o=['bio_o2o','bio_op_bob2','bio_op_sumo2', 'bee_analytics_transporte_pipeline_o2o','bee_analytics_transporte_presal']
df_o2o_day = df_d0.loc[(df_d0['dataset_id'].isin(o2o)) & (df_d0['qtde_execucao_dia'].notnull()) ]

df_o2o_day = df_o2o_day[['dataset_id','table_id','prefix_projeto','qtde_execucao_dia', 'custo_ano_bee','custo_dia', 'custo_dia_mes','ulti_data_hora_execucao_job_dia']].sort_values('custo_dia_mes', ascending=False)
df_o2o_day.reset_index(inplace=True , drop=True)#
#df_o2o_day
#
#df_o2o_day.groupby(['table_id'])['custo_dia_mes'].max()


In [34]:
#tabela com maior custo mes
df_o2o_day[df_o2o_day['custo_dia_mes']==df_o2o_day['custo_dia_mes'].max()]

,dataset_id,table_id,prefix_projeto,qtde_execucao_dia,custo_ano_bee,custo_dia,custo_dia_mes,ulti_data_hora_execucao_job_dia
0,bio_op_bob2,bob2_pedido_de_venda_detalhes,o2o,9,321.9,1.21,36.3,2023-07-24 11:00:34.528


In [35]:
#CUSTO DETALHE DO GRUPO REPORTS O2O
total_day = df_o2o_day[['custo_dia','custo_dia_mes']].sum()

print(f'total em $:\n{total_day}\n')
print(f'total em R$:\n{total_day*5}\n')
print(f'total em R$ no ano:\n{(total_day*5)*12}\n')

#ANALISE EM RELAÇÃO AO PERCENTUAL DE FATIA DO CUSTO DO PROJETO EM COMPARAÇÃO COM GERAL
reports = df_o2o_day['custo_dia'].sum()
porc = (reports*100)/df_d0['custo_dia'].sum()

print(f'custo do PIPELINE O2O representa {int(porc)}% do custo geral dos jobs executados no dia')

#SOMA REPORTS + PIPELINE
o2o = df_o2o_day['custo_dia'].sum()+df_day_reports['custo_dia'].sum()
porc = (o2o*100)/df_d0['custo_dia'].sum()
print(f'custo do PIPELINE O2O + REPORTS O2O representa {int(porc)}% do custo geral dos jobs executados no dia')

total em $:
custo_dia         3.19
custo_dia_mes    95.70
dtype: float64

total em R$:
custo_dia         15.95
custo_dia_mes    478.50
dtype: float64

total em R$ no ano:
custo_dia         191.4
custo_dia_mes    5742.0
dtype: float64

custo do PIPELINE O2O representa 32% do custo geral dos jobs executados no dia
custo do PIPELINE O2O + REPORTS O2O representa 69% do custo geral dos jobs executados no dia


In [36]:
#INSTANCIA FORÇADA

iff=['bio_op_instancia_forcada']
df_iff_day = df_d0.loc[(df_d0['dataset_id'].isin(iff)) & (df_d0['qtde_execucao_dia'].notnull()) ]

df_iff_day = df_iff_day[['dataset_id','table_id','prefix_projeto','qtde_execucao_dia', 'custo_ano_bee','custo_dia', 'custo_dia_mes','ulti_data_hora_execucao_job_dia']].sort_values('custo_dia_mes', ascending=False)
df_iff_day.reset_index(inplace=True , drop=True)#
#df_o2o_day
#
#df_o2o_day.groupby(['table_id'])['custo_dia_mes'].max()

In [37]:
#tabela com maior custo mes
df_iff_day[df_iff_day['custo_dia_mes']==df_iff_day['custo_dia_mes'].max()]

,dataset_id,table_id,prefix_projeto,qtde_execucao_dia,custo_ano_bee,custo_dia,custo_dia_mes,ulti_data_hora_execucao_job_dia
0,bio_op_instancia_forcada,instancia_forcada_delivery,instancia_forçada,1,0.61,1.54,46.2,2023-07-24 06:00:07.327


In [38]:
#CUSTO DETALHE DO GRUPO REPORTS O2O
total_day = df_iff_day[['custo_dia','custo_dia_mes']].sum()

print(f'total em $:\n{total_day}\n')
print(f'total em R$:\n{total_day*5}\n')
print(f'total em R$ no ano:\n{(total_day*5)*12}\n')

#ANALISE EM RELAÇÃO AO PERCENTUAL DE FATIA DO CUSTO DO PROJETO EM COMPARAÇÃO COM GERAL
reports = df_iff_day['custo_dia'].sum()
porc = (reports*100)/df_d0['custo_dia'].sum()

print(f'custo do projeto IF representa {int(porc)}% do custo geral dos jobs executados no dia')


total em $:
custo_dia         1.54
custo_dia_mes    46.20
dtype: float64

total em R$:
custo_dia          7.7
custo_dia_mes    231.0
dtype: float64

total em R$ no ano:
custo_dia          92.4
custo_dia_mes    2772.0
dtype: float64

custo do projeto IF representa 15% do custo geral dos jobs executados no dia


In [39]:
# # gera uma lista de todas as tabelas geradas no df dos datasets
# list_tab = [str(l) for l in df_day_reports['table_id'].tolist()]

In [40]:
#MONITORIA

monitoria=['bee_analytics_transporte_monitoria','bee_analytics_transporte']
df_monitoria_day = df_d0.loc[(df_d0['dataset_id'].isin(monitoria)) & (df_d0['qtde_execucao_dia'].notnull()) ]

df_monitoria_day = df_monitoria_day[['dataset_id','table_id','prefix_projeto','qtde_execucao_dia', 'custo_ano_bee','custo_dia', 'custo_dia_mes','ulti_data_hora_execucao_job_dia']].sort_values('custo_dia_mes', ascending=False)
df_monitoria_day.reset_index(inplace=True , drop=True)#
#df_o2o_day
#
#df_o2o_day.groupby(['table_id'])['custo_dia_mes'].max()

In [41]:
#tabela com maior custo mes
df_monitoria_day[df_monitoria_day['custo_dia_mes']==df_monitoria_day['custo_dia_mes'].max()]

,dataset_id,table_id,prefix_projeto,qtde_execucao_dia,custo_ano_bee,custo_dia,custo_dia_mes,ulti_data_hora_execucao_job_dia
0,bee_analytics_transporte,information_executor_jobs,monitoria,6,0.4,1.13,33.9,2023-07-24 11:10:34.985


In [42]:
#CUSTO DETALHE DO GRUPO REPORTS O2O
total_day = df_monitoria_day[['custo_dia','custo_dia_mes']].sum()

print(f'total em $:\n{total_day}\n')
print(f'total em R$:\n{total_day*5}\n')
print(f'total em R$ no ano:\n{(total_day*5)*12}\n')

#ANALISE EM RELAÇÃO AO PERCENTUAL DE FATIA DO CUSTO DO PROJETO EM COMPARAÇÃO COM GERAL
reports = df_monitoria_day['custo_dia'].sum()
porc = (reports*100)/df_d0['custo_dia'].sum()

print(f'custo do projeto MONITORIA representa {int(porc)}% do custo geral dos jobs executados no dia')

total em $:
custo_dia         1.43
custo_dia_mes    42.90
dtype: float64

total em R$:
custo_dia          7.15
custo_dia_mes    214.50
dtype: float64

total em R$ no ano:
custo_dia          85.8
custo_dia_mes    2574.0
dtype: float64

custo do projeto MONITORIA representa 14% do custo geral dos jobs executados no dia
